# MediaPipe

1. 실시간 손 인식 및 추적

In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
hands = mp_hands.Hands() # 손 추적 객체 생성

mp_drawing = mp.solutions.drawing_utils

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    frame = cv2.flip(frame, 1)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    results = hands.process(img_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks, hand_lr in zip(results.multi_hand_landmarks, results.multi_handedness):
            label = hand_lr.classification[0].label
            
            img_h, img_w, _ = frame.shape
            
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            if label == 'Left':
                cv2.putText(frame, 'Left', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, 'Right', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
    cv2.imshow('Hand Tracking', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

### 2. 손가락 사이 거리 측정

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    frame = cv2.flip(frame, 1)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    results = hands.process(img_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks, hand_lr in zip(results.multi_hand_landmarks, results.multi_handedness):
            
            #손목 좌표
            wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]

            label = hand_lr.classification[0].label
            
            img_h, img_w, _ = frame.shape
            
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            if label == 'Left':
                cv2.putText(frame, 'Left', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif label == 'Right':
                cv2.putText(frame, 'Right', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            if label == 'Right':
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]    #THUMB_TIP 엄지 손가락 좌표
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]  # INDEX_FINGER_TIP 검지 손가락 좌표
                
                thumb_x = thumb_tip.x*img_w
                thumb_y = thumb_tip.y*img_h
                idx_finger_x = index_finger_tip.x*img_w 
                idx_finger_y = index_finger_tip.y*img_h
                
                point1 = np.array([thumb_x, thumb_y])
                point2 = np.array([idx_finger_x,idx_finger_y])

                distance = np.linalg.norm(point2 - point1)  # 유클리드 거리 계산
                
                cv2.putText(frame,str(int(distance)),(300,100),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
                
    cv2.imshow('Hand Tracking', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

### 3. 손가락 관절 각도 계산

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a-b
    bc = c-b

    cosangle = np.dot(ba,bc)/(np.linalg.norm(ba)*np.linalg.norm(bc))
    arccos = np.arccos(cosangle)
    return np.degrees(arccos)

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    frame = cv2.flip(frame, 1)      # 좌우반전
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks, hand_lr in zip(results.multi_hand_landmarks, results.multi_handedness):
                        
            if1 = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
            if2 = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]
            if3 = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP]

            if1_list = [if1.x, if1.y, if1.z]
            if2_list = [if2.x, if2.y, if2.z]
            if3_list = [if3.x, if3.y, if3.z]
            index_finger_degree = calculate_angle(if1_list, if2_list, if3_list)

            if not np.isnan(index_finger_degree):
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                cv2.putText(frame, f'Index Finger Angle: {int(index_finger_degree)}', (10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
            

    cv2.imshow('hand', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'calculate_angle' is not defined

: 